# Coronavirus Data Analysis

In [1]:
import pandas as pd
import numpy as np

In [2]:
spark.sparkContext.applicationId

'application_1636170550178_0012'

In [3]:
pd.options.display.max_columns = 35

### Create Variables for Queries

In [4]:
# Look back days variables
daysInterval = 365
print(daysInterval)
maxDays = 365
print(maxDays)
oneYearInterval=365
print(oneYearInterval)

# Username variable
user_id = 'rc047407'           ####<-------------------- change to your user id (Ex: tl068507)
userName = 'user_' + user_id
print(userName)

# Algorithm name variable
algorithmName='CoronavirusLengthOfStay_HighLevel'
print(algorithmName)

365
365
365
user_rc047407
CoronavirusLengthOfStay_HighLevel


In [5]:
stdVar = spark.sql(f"""
  select  round(std(h.length_of_stay),2) as std_length_of_stay
       
  from {userName}.{algorithmName}_Hospital h
  
  left join {userName}.{algorithmName}_Vaccine v
    on h.personid = v.personId
      and h.tenant = v.tenant

   where    
      datediff(h.serviceDate, h.dischargeDate) < 0
      and h.length_of_stay > 0
       
""")
stdValue = stdVar.first()['std_length_of_stay']
print(stdValue)

12.8


In [6]:
stdVar = spark.sql(f"""
  select  round(std(h.length_of_stay),2) as std_length_of_stay
       
  from {userName}.{algorithmName}_Hospital h
  
  left join {userName}.{algorithmName}_Vaccine v
    on h.personid = v.personId
      and h.tenant = v.tenant

   where    
      datediff(h.serviceDate, h.dischargeDate) < 0
      and h.length_of_stay > 0
 
      and  ( datediff(h.serviceDate, v.vaccine_date) <= 0
             or v.vaccine_date is null) 
       
""")
stdValue_nonVac = stdVar.first()['std_length_of_stay']
print(stdValue_nonVac)

12.81


In [7]:
stdVar = spark.sql(f"""
  select  round(std(h.length_of_stay),2) as std_length_of_stay
       
  from {userName}.{algorithmName}_Hospital h
  
  left join {userName}.{algorithmName}_Vaccine v
    on h.personid = v.personId
      and h.tenant = v.tenant

   where    
      datediff(h.serviceDate, h.dischargeDate) < 0
      and h.length_of_stay > 0
               
      and  ( datediff(h.serviceDate, v.vaccine_date) > 0
             and v.vaccine_date is not null)        
      
    
""")
stdValue_vac = stdVar.first()['std_length_of_stay']
print(stdValue_vac)

5.62


In [8]:
spark.sql(f"""

          select
              case
                    when discharged_to_skilled_nursing_facility = 1 then "Skilled Nursing Facility"
                    when discharged_to_hospice = 1 then 'Hospice Care'
                    when died_in_the_hospital = 1 then 'Mortuary'
                    when discharged_to_rehab_facility = 1 then 'Rehab Facility'
                    when discharged_to_long_term_care_facility = 1 then 'Long Term Care Facility'
                else 'Home'
                end as discharge_to
              , count(h.personId) as count_of_non_vaccinated_patients
              , round(mean(h.length_of_stay),2) as mean_length_of_stay
        from {userName}.{algorithmName}_Hospital h
          
        left join {userName}.{algorithmName}_Vaccine v
            on h.personid = v.personId
              and h.tenant = v.tenant

           where    
              datediff(h.serviceDate, h.dischargeDate) < 0
              and h.length_of_stay > 0
              and h.length_of_stay < 2*{stdValue_nonVac}

              and  ( datediff(h.serviceDate, v.vaccine_date) <= 0
                     or v.vaccine_date is null) 
               
        group by 1
        order by 1
     
""").toPandas()

,discharge_to,count_of_non_vaccinated_patients,mean_length_of_stay
0,Home,216584,6.69
1,Hospice Care,6994,9.36
2,Long Term Care Facility,1925,13.36
3,Mortuary,37663,11.20
4,Rehab Facility,3107,11.98
5,Skilled Nursing Facility,27034,10.00


In [9]:
spark.sql(f"""

        select count(h.personId) as count_of_non_vaccinated_patients
               , round(mean(h.length_of_stay),2) as mean_length_of_stay
        from {userName}.{algorithmName}_Hospital h
          
        left join {userName}.{algorithmName}_Vaccine v
            on h.personid = v.personId
              and h.tenant = v.tenant

           where    
              datediff(h.serviceDate, h.dischargeDate) < 0
              and h.length_of_stay > 0
              and h.length_of_stay < 2*{stdValue_nonVac}

              and length(h.dischargeDate) > 3
              and length(h.serviceDate) > 3


              and  ( datediff(h.serviceDate, v.vaccine_date) <= 0
                    or v.vaccine_date is not null)  

""").toPandas()

,count_of_non_vaccinated_patients,mean_length_of_stay
0,8092,7.15


In [10]:
spark.sql(f"""

          select
                 case
                    when discharged_to_skilled_nursing_facility = 1 then "Skilled Nursing Facility"
                    when discharged_to_hospice = 1 then 'Hospice Care'
                    when died_in_the_hospital = 1 then 'Mortuary'
                    when discharged_to_rehab_facility = 1 then 'Rehab Facility'
                    when discharged_to_long_term_care_facility = 1 then 'Long Term Care Facility'
                else 'Home'
                end as discharge_to
              , count(h.personId) as count_of_vaccinated_patients
              , round(mean(h.length_of_stay),2) as mean_length_of_stay
        from {userName}.{algorithmName}_Hospital h
          
        left join {userName}.{algorithmName}_Vaccine v
            on h.personid = v.personId
              and h.tenant = v.tenant

           where    
              datediff(h.serviceDate, h.dischargeDate) < 0
              and h.length_of_stay > 0
              and h.length_of_stay < 2*{stdValue_vac}

              and  ( datediff(h.serviceDate, v.vaccine_date) > 0
                    and v.vaccine_date is not null)  
               
        group by 1
        order by 1
     
""").toPandas()

,discharge_to,count_of_vaccinated_patients,mean_length_of_stay
0,Home,295,5.26
1,Hospice Care,11,6.00
2,Mortuary,15,4.60
3,Skilled Nursing Facility,18,5.61


In [11]:
spark.sql(f"""

          select count(h.personId) as count_of_vaccinated_patients
                , round(mean(h.length_of_stay),2) as mean_length_of_stay
        from {userName}.{algorithmName}_Hospital h
          
        left join {userName}.{algorithmName}_Vaccine v
            on h.personid = v.personId
              and h.tenant = v.tenant

           where    
              datediff(h.serviceDate, h.dischargeDate) < 0
              and h.length_of_stay > 0
              and h.length_of_stay < 2*{stdValue_vac}

              and length(h.dischargeDate) > 3
              and length(h.serviceDate) > 3


              and  ( datediff(h.serviceDate, v.vaccine_date) > 0
                    and v.vaccine_date is not null)  
               
      
""").toPandas()

,count_of_vaccinated_patients,mean_length_of_stay
0,339,5.28


In [12]:
spark.sql(f"""

          select
                case
                    when discharged_to_skilled_nursing_facility = 1 then "Skilled Nursing Facility"
                    when discharged_to_hospice = 1 then 'Hospice Care'
                    when died_in_the_hospital = 1 then 'Mortuary'
                    when discharged_to_rehab_facility = 1 then 'Rehab Facility'
                    when discharged_to_long_term_care_facility = 1 then 'Long Term Care Facility'
                else 'Home'
                end as discharge_to
              , count(h.personId) as count_of_all_patients
              , round(mean(h.length_of_stay),2) as mean_length_of_stay
        from {userName}.{algorithmName}_Hospital h
          
        left join {userName}.{algorithmName}_Vaccine v
            on h.personid = v.personId
              and h.tenant = v.tenant

           where    
              datediff(h.serviceDate, h.dischargeDate) < 0
              and h.length_of_stay > 0
              and h.length_of_stay < 2*{stdValue}

        group by 1
        order by 1
     
""").toPandas()

,discharge_to,count_of_all_patients,mean_length_of_stay
0,Home,216915,6.69
1,Hospice Care,7013,9.37
2,Long Term Care Facility,1926,13.36
3,Mortuary,37695,11.20
4,Rehab Facility,3110,11.99
5,Skilled Nursing Facility,27056,9.99


In [13]:
spark.sql(f"""

        select count(h.personId) as count_of_non_vaccinated_patients
              , round(mean(h.length_of_stay),2) as mean_length_of_stay
        from {userName}.{algorithmName}_Hospital h
          
        left join {userName}.{algorithmName}_Vaccine v
            on h.personid = v.personId
              and h.tenant = v.tenant

           where    
              datediff(h.serviceDate, h.dischargeDate) < 0
              and h.length_of_stay > 0
              and h.length_of_stay < 2*{stdValue}

              and length(h.dischargeDate) > 3
              and length(h.serviceDate) > 3

     
""").toPandas()

,count_of_non_vaccinated_patients,mean_length_of_stay
0,293715,7.74
